In [ ]:
import sys
sys.path.append('../src')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO

from utils import (
    load_image,
    extract_edge_features,
    detect_keypoints_sift,
    detect_keypoints_orb,
    detect_contours,
    draw_detections,
    draw_enhanced_detections
)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 3.1 Load Image and Model

In [ ]:
# Load test image
image_path = "../data/raw/my_dogs.jpg"

if not Path(image_path).exists():
    print("⚠️  Using sample image")
    image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
else:
    image = load_image(image_path)

# Load YOLO model
model_path = "../models/yolov8n-pets.pt"
if Path(model_path).exists():
    model = YOLO(model_path)
    print(f"✅ Model loaded from {model_path}")
else:
    print(f"❌ Model not found at {model_path}")
    print("   Please download or train a YOLOv8 model first")

print(f"Image shape: {image.shape}")

## 3.2 Extract Features

In [ ]:
# Extract edge features
edges = extract_edge_features(image)

# Detect keypoints
keypoints_sift, descriptors_sift = detect_keypoints_sift(image, max_keypoints=200)
keypoints_orb, descriptors_orb = detect_keypoints_orb(image, max_keypoints=200)

# Detect contours
contours = detect_contours(image, min_area=500)

print(f"📊 Feature Extraction Results:")
print(f"  • Canny edges: {edges['canny'].sum() // 255} pixels")
print(f"  • SIFT keypoints: {len(keypoints_sift)}")
print(f"  • ORB keypoints: {len(keypoints_orb)}")
print(f"  • Contours: {len(contours)}")

## 3.3 Visualize Edge Detection Results

In [ ]:
# Convert for display
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].imshow(image_rgb)
axes[0, 0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(edges['canny'], cmap='gray')
axes[0, 1].set_title('Canny Edge Detection', fontsize=12)
axes[0, 1].axis('off')

axes[1, 0].imshow(edges['sobel'], cmap='gray')
axes[1, 0].set_title('Sobel Edge Detection', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(edges['laplacian'], cmap='gray')
axes[1, 1].set_title('Laplacian Edge Detection', fontsize=12)
axes[1, 1].axis('off')

plt.suptitle('Edge Detection Methods', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 3.4 Run YOLO Detection

In [ ]:
# Run inference
results = model(image, conf=0.25, verbose=False)

# Extract results
result = results[0]
boxes = result.boxes.xyxy.cpu().numpy()
confidences = result.boxes.conf.cpu().numpy()
class_ids = result.boxes.cls.cpu().numpy().astype(int)
class_names = result.names

print(f"\n🐾 Detected {len(boxes)} pet(s):")
for i, (box, class_id, conf) in enumerate(zip(boxes, class_ids, confidences)):
    class_name = class_names[class_id]
    print(f"  {i+1}. {class_name.upper()} - confidence: {conf:.2f}")

## 3.5 Compare Standard vs Enhanced Detection

In [ ]:
# Standard YOLO detection
standard_output = draw_detections(
    image,
    boxes.tolist(),
    class_names,
    confidences.tolist(),
    class_ids.tolist()
)

# Enhanced detection with edges
enhanced_edges = draw_enhanced_detections(
    image,
    boxes.tolist(),
    class_names,
    confidences.tolist(),
    class_ids.tolist(),
    show_edges=True,
    show_keypoints=False
)

# Enhanced detection with keypoints
enhanced_keypoints = draw_enhanced_detections(
    image,
    boxes.tolist(),
    class_names,
    confidences.tolist(),
    class_ids.tolist(),
    show_edges=False,
    show_keypoints=True
)

# Enhanced detection with both
enhanced_both = draw_enhanced_detections(
    image,
    boxes.tolist(),
    class_names,
    confidences.tolist(),
    class_ids.tolist(),
    show_edges=True,
    show_keypoints=True
)

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].imshow(cv2.cvtColor(standard_output, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Standard YOLO Detection', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(enhanced_edges, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('YOLO + Edge Detection', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(cv2.cvtColor(enhanced_keypoints, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('YOLO + Keypoints', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(enhanced_both, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('YOLO + Edges + Keypoints', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

plt.suptitle('Detection Method Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 3.6 Analyze Detection Regions

In [ ]:
# For each detection, analyze the features within the bounding box
for i, (box, class_id, conf) in enumerate(zip(boxes, class_ids, confidences)):
    x1, y1, x2, y2 = map(int, box)
    class_name = class_names[class_id]
    
    # Extract region
    region = image[y1:y2, x1:x2]
    
    # Analyze features in region
    region_edges = extract_edge_features(region)
    region_keypoints_orb, _ = detect_keypoints_orb(region, max_keypoints=100)
    
    print(f"\n{i+1}. {class_name.upper()} Detection Analysis:")
    print(f"   Bounding Box: [{x1}, {y1}] to [{x2}, {y2}]")
    print(f"   Region size: {x2-x1} x {y2-y1} pixels")
    print(f"   Edge pixels (Canny): {region_edges['canny'].sum() // 255}")
    print(f"   ORB keypoints in region: {len(region_keypoints_orb)}")
    print(f"   Confidence: {conf:.2%}")

## Summary

**What we accomplished:**
- ✅ Integrated YOLOv8 object detection with edge detection
- ✅ Combined multiple computer vision techniques
- ✅ Created enhanced visualization options
- ✅ Analyzed feature distributions in detected regions

**Use cases:**
- Enhanced visual analysis for debugging detection issues
- Feature-rich representations for downstream tasks
- Understanding what the model "sees" in detected regions

**Next steps:**
- Fine-tune YOLO on custom pet dataset
- Add tracking for video streams
- Deploy as a web service